In [20]:
import pandas as pd 
import numpy as np

d= pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

In [21]:
df =d[0]

Combine multiple columns into one column using stack

In [22]:
df2 = df.stack().reset_index()


In [23]:
df2 = df2[df2.columns[2]]
type(df2)

pandas.core.series.Series

In [24]:
df2 = df2.to_frame()
df2.head()

,0
0,M1ANot assigned
1,M2ANot assigned
2,M3ANorth York(Parkwoods)
3,M4ANorth York(Victoria Village)
4,M5ADowntown Toronto(Regent Park / Harbourfront)


Extract postal code 

In [25]:
#add in postal code 
df2['PostalCode'] = df2[df2.columns[0]].astype(str).str[0:3]

In [26]:
df2.columns = ['overall', 'PostalCode']

In [27]:
df2.columns

Index(['overall', 'PostalCode'], dtype='object')

remove unassigned data

In [28]:
df2 = df2[~df2.overall.str.contains("Not assigned",na=False)]

In [29]:
s = df2[df2.columns[0]]
type(s)

pandas.core.series.Series

In [30]:
df2['Neighborhood'] = s.apply(lambda st: st[st.find("(")+1:st.find(")")])

In [31]:
df2['test'] = df2['overall'].str.replace(r"\(.*\)","")

df2['Borough'] = df2['test'].astype(str).str[3:]

In [32]:
df2= df2.drop(columns = ['overall','test'])

In [33]:
df2

,PostalCode,Neighborhood,Borough
2,M3A,Parkwoods,North York
3,M4A,Victoria Village,North York
4,M5A,Regent Park / Harbourfront,Downtown Toronto
5,M6A,Lawrence Manor / Lawrence Heights,North York
6,M7A,Queen's Park / Ontario Provincial Government,Downtown Toronto
8,M9A,Islington Avenue,Etobicoke
9,M1B,Malvern / Rouge,Scarborough
11,M3B,Don Mills,North YorkNorth
12,M4B,Parkview Hill / Woodbine Gardens,East York
13,M5B,"Garden District, Ryerson",Downtown Toronto


In [34]:
df2.shape

(103, 3)

In [35]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

NameError: name 'postal_code' is not defined

In [51]:
Geospatial = pd.read_csv('Geospatial_Coordinates.csv')
Geospatial.head()
Geospatial = Geospatial.rename(columns={"Postal Code": "PostalCode"})


In [52]:
df2.head()

,PostalCode,Neighborhood,Borough
2,M3A,Parkwoods,North York
3,M4A,Victoria Village,North York
4,M5A,Regent Park / Harbourfront,Downtown Toronto
5,M6A,Lawrence Manor / Lawrence Heights,North York
6,M7A,Queen's Park / Ontario Provincial Government,Downtown Toronto


In [53]:
Geospatial.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [54]:
Geo = pd.merge(df2, Geospatial, on ='PostalCode', how = 'left')
Geo.head()

,PostalCode,Neighborhood,Borough,Latitude,Longitude
0,M3A,Parkwoods,North York,43.753259,-79.329656
1,M4A,Victoria Village,North York,43.725882,-79.315572
2,M5A,Regent Park / Harbourfront,Downtown Toronto,43.654260,-79.360636
3,M6A,Lawrence Manor / Lawrence Heights,North York,43.718518,-79.464763
4,M7A,Queen's Park / Ontario Provincial Government,Downtown Toronto,43.662301,-79.389494
